# Material ID : mp-3056

In [1]:
import numpy as np
from mp_api.client import MPRester
from pymatgen.core.operations import SymmOp
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.electronic_structure.plotter import BSPlotter
from pymatgen.phonon.plotter import PhononBSPlotter
from jupyter_jsmol.pymatgen import quick_view
from lmapr1492 import plot_brillouin_zone, get_plot_bs, get_plot_dos, get_plot_bs_and_dos, get_branch_wavevectors
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from pymatgen.core.lattice import Lattice
from jupyter_jsmol import JsmolView
from ipywidgets import Layout, widgets, interact
from IPython.display import display
import json
# An optional utility to display a progress bar
# for long-running loops. `pip install tqdm`.
from tqdm import tqdm

In [2]:
api_key = "IDG8oerg7OP5AeRfbNjUZBb1Ad4Qke2l"
material_id = "mp-3056"

mpr = MPRester(api_key)

structure = mpr.get_structure_by_material_id(material_id)
el_bs = mpr.get_bandstructure_by_material_id(material_id)
el_dos = mpr.get_dos_by_material_id(material_id)
ph_bs = mpr.get_phonon_bandstructure_by_material_id(material_id)
ph_dos = mpr.get_phonon_dos_by_material_id(material_id)

conv_struc = SpacegroupAnalyzer(structure).get_conventional_standard_structure()
symmops = SpacegroupAnalyzer(conv_struc).get_space_group_operations()

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving PhononBSDOSDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving PhononBSDOSDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

# Télécharger le fichier .cif

In [3]:
from pymatgen.io.cif import CifWriter

CifWriter(structure).write_file(material_id+".cif")

## Représentation NaTlO2 dans la maille conventionnelle.  

In [4]:
view = quick_view(conv_struc, "packed")
display(view)
view.script('rotate on')

JsmolView(layout=Layout(align_self='stretch', height='400px'))

# Étude de la symmétrie

In [5]:
#Création d'un objet SpaceGroupAnalyzer
struct = SpacegroupAnalyzer(structure)
new_s  = SpacegroupAnalyzer(structure).get_primitive_standard_structure()
space_group = SpacegroupAnalyzer(new_s).get_space_group_symbol()
print("Molecule formula :",new_s.reduced_formula,"\t \t space group :", space_group)

Molecule formula : NaTlO2 	 	 space group : R-3m


In [6]:
#récupération de la structure conventionnel qui correspond à la maille du fichier CIF symmetrized
struct_analyze = SpacegroupAnalyzer(conv_struc)

view = quick_view(structure, conventional=True)#, supercell=[2,2,1])
display(view)

JsmolView(layout=Layout(align_self='stretch', height='400px'))

In [7]:
view.script("select all; label on")

In [8]:
# récupération des opération de symmétrie
sym_op = struct_analyze.get_symmetry_operations()
"""
for i in range(len(sym_op)):
    print("----", i, "----\n", sym_op[i])
"""
print(sym_op[12]) #translation (tau [0.66666667 0.33333333 0.33333333])
print("__________")
print(sym_op[6])  #rot ??
print("__________")
print(sym_op[16]) #rot ?? + translation (tau [0.66666667 0.33333333 0.33333333])

#récupération de tous les sites d'atome dans la maille cristalline
sites = conv_struc.sites 
print("\n",sites)

Rot:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
tau
[0.66666667 0.33333333 0.33333333]
__________
Rot:
[[ 0.  1.  0.]
 [ 1.  0.  0.]
 [ 0.  0. -1.]]
tau
[0. 0. 0.]
__________
Rot:
[[-1.  1.  0.]
 [-1.  0.  0.]
 [ 0.  0.  1.]]
tau
[0.66666667 0.33333333 0.33333333]

 [PeriodicSite: Na (1.684, 0.9724, 2.728) [0.3333, 0.6667, 0.1667], PeriodicSite: Na (3.369, 0.0, 8.183) [1.0, 1.0, 0.5], PeriodicSite: Na (1.684, -0.9724, 13.64) [0.6667, 0.3333, 0.8333], PeriodicSite: Tl (0.0, 0.0, 0.0) [0.0, 0.0, 0.0], PeriodicSite: Tl (1.684, -0.9724, 5.456) [0.6667, 0.3333, 0.3333], PeriodicSite: Tl (1.684, 0.9724, 10.91) [0.3333, 0.6667, 0.6667], PeriodicSite: O (1.684, -0.9724, 1.24) [0.6667, 0.3333, 0.07579], PeriodicSite: O (0.0, 0.0, 4.215) [0.0, 0.0, 0.2575], PeriodicSite: O (1.684, 0.9724, 6.696) [0.3333, 0.6667, 0.4091], PeriodicSite: O (1.684, -0.9724, 9.671) [0.6667, 0.3333, 0.5909], PeriodicSite: O (0.0, 0.0, 12.15) [0.0, 0.0, 0.7425], PeriodicSite: O (1.684, 0.9724, 15.13) [0.3333, 0.6667, 0.9242]

# Premier élément de symétrie
Le premier élément de symétrie est une translation dans la direction $<1 1 1>$.

La matrice qui représente cette opération de symétrie est la suivante : $$\left({\begin{array}{cc}
    1 & 0 & 0 \\
    0 & 1 & 0 \\
    0 & 0 & 1 \\
  \end{array} }\right)$$, c'est-à-dire la matrice identité à laquelle on ajoute une translation $(0.6667, 0.3333, 0.3333)$.
  
C'est l'opération renseignée à la position 12 de la liste que l'on retrouve plus haut.

L'équation est donc : $$\begin{pmatrix}  1 & 0 & 0 \\
    0 & 1 & 0 \\
    0 & 0 & 1 \end{pmatrix} 
\begin{pmatrix}  
x \\ 
y \\ 
z 
\end{pmatrix} + \begin{pmatrix}
    0.6667 \\
    0.3333 \\
    0.3333\\
  \end{pmatrix} = \begin{pmatrix}
   x+0.6667\\
   y+0.3333 \\
   z+0.3333 \\
  \end{pmatrix}$$
  
Cet élément de symétrie va être appliqué à l'atome $\textit{Na}$ dont les coordonnées réduites sont : $(0.3333, 0.6667, 0.1667)$

In [9]:
op1 = sym_op[12]
print(op1)
Na1 = sites[0]
atome = str(Na1.species)

print("\nCoordonnées réduites avant l'opération de symétrie sur l'atome de", atome[:2], " : ", Na1.frac_coords)
# application de l'opérateur de symétrie
Na1_afterop = op1.operate(Na1.frac_coords)
print("Coordonnées réduites après l'opération :", Na1_afterop)

Rot:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
tau
[0.66666667 0.33333333 0.33333333]

Coordonnées réduites avant l'opération de symétrie sur l'atome de Na  :  [0.33333333 0.66666667 0.16666667]
Coordonnées réduites après l'opération : [1.  1.  0.5]


In [10]:
# représentation graphique
view = quick_view(conv_struc,supercell=[2,1,1])
display(view)
view.script('draw SYMOP 13 {atomno = 1}; select atomno=1; label on')

JsmolView(layout=Layout(align_self='stretch', height='400px'))

# Deuxième élément de symétrie
Le deuxième élément de symétrie est une rotation d'ordre 2 dans la direction $<1 1 0>$.

La matrice qui représente cette opération de symétrie est la suivante : $$\left({\begin{array}{cc}
    0 & 1 & 0 \\
    1 & 0 & 0 \\
    0 & 0 & -1 \\
  \end{array} }\right)$$
  
C'est l'opération renseignée à la position 6 de la liste que l'on retrouve plus haut.
Cet élément de symétrie va être appliqué à l'atome $\textit{Na}$ dont les coordonnées réduites sont : $(0.3333, 0.6667, 0.1667)$

In [11]:
op2 = sym_op[6]
print(op2)
Na2 = sites[0]
atome = str(Na2.species)

print("\nCoordonnées réduites avant l'opération de symétrie sur l'atome de", atome[:2], " : ", Na2.frac_coords)
# application de l'opérateur de symétrie
Na2_afterop = op2.operate(Na2.frac_coords)
print("Coordonnées réduites après l'opération :", Na2_afterop)

Rot:
[[ 0.  1.  0.]
 [ 1.  0.  0.]
 [ 0.  0. -1.]]
tau
[0. 0. 0.]

Coordonnées réduites avant l'opération de symétrie sur l'atome de Na  :  [0.33333333 0.66666667 0.16666667]
Coordonnées réduites après l'opération : [ 0.66666667  0.33333333 -0.16666667]


In [12]:
# représentation graphique
view = quick_view(conv_struc,supercell=[1,1,-1])
display(view)
view.script('draw SYMOP 7 {atomno = 1}; select atomno=1; label on')

JsmolView(layout=Layout(align_self='stretch', height='400px'))

# Troisième élément de symétrie
Le troisième élément de symétrie est une roto-inversion d'ordre $6$ dans la direction $<2 1 0>$. 

La matrice qui représente la rotation de cette opération de symétrie est la suivante : $$\begin{pmatrix}
    -1 & 1 & 0 \\
    -1 & 0 & 0 \\
    0 & 0 & 1
  \end{pmatrix}$$
Pour compléter l'opération de symmétrie, il faut appliquer une translation $(0.6667, 0.3333, 0.3333)$.
  
L'équation est donc : $$\begin{pmatrix} -1 & 1 & 0 \\
    -1 & 0 & 0 \\
    0 & 0 & 1  \end{pmatrix} 
\begin{pmatrix}  
x \\ 
y \\ 
z 
\end{pmatrix} + \begin{pmatrix}
    0.6667 \\
    0.3333 \\
    0.3333 \\
  \end{pmatrix} = \begin{pmatrix}
   -x+y+0.6667 \\
   -x+0.3333 \\
   z+0.3333 \\
  \end{pmatrix}$$
  
C'est l'opération renseignée à la position 16 de la liste que l'on retrouve plus haut. Cet élément de symétrie va être appliqué à l'atome de $\textit{Na}$ dont les coordonnées réduites sont : $(0.6667, 0.3333, 0.3333)$.

In [13]:
op3 = sym_op[16]
print(op3)
Na3 = sites[2]
atome = str(Na3.species)

print("\nCoordonnées réduites avant l'opération de symétrie sur l'atome de", atome[:2], " : ", Na3.frac_coords)
# application de l'opérateur de symétrie
Na3_afterop = op3.operate(Na3.frac_coords)
print("Coordonnées réduites après l'opération :", Na3_afterop)

Rot:
[[-1.  1.  0.]
 [-1.  0.  0.]
 [ 0.  0.  1.]]
tau
[0.66666667 0.33333333 0.33333333]

Coordonnées réduites avant l'opération de symétrie sur l'atome de Na  :  [0.66666667 0.33333333 0.83333333]
Coordonnées réduites après l'opération : [ 0.33333333 -0.33333333  1.16666667]


In [14]:
# représentation graphique
view = quick_view(conv_struc,supercell=[1,1,1])
display(view)
view.script('draw SYMOP 17 {atomno = 20}; select atomno=20; label on')

JsmolView(layout=Layout(align_self='stretch', height='400px'))

In [15]:
view.script('draw SYMOP 17 {atomno = 20}; select atomno=20; label on')